# Чтение содержимого файлов

In [1]:
import pandas as pd

## МС_Владимирская область.xls

In [3]:
df = pd.read_excel('Marketing_Lists\МС_Владимирская область.xls')
print(df.head(10))

   ID Находится в реестре МСП Размер компании.Наименование  \
0   1                     Нет                          NaN   
1   2                     Нет                          NaN   
2   3                     Нет                          NaN   
3   4                     Нет                          NaN   
4   5                     Нет                          NaN   
5   6                     Нет                          NaN   
6   7                     Нет                          NaN   
7   8                     Нет                          NaN   
8   9                     Нет                  Микробизнес   
9   9                     Нет                  Микробизнес   

   Размер уставного капитала объявленный  \
0                                    NaN   
1                                    NaN   
2                                    NaN   
3                                    NaN   
4                                    NaN   
5                                    NaN   
6        

## «ЕЛС действующий» – единый лицевой счёт клиента, используемый для оплаты услуг ОАО «РЖД».
Критерий: да/нет («да» – у клиента есть ЕЛС, есть заключенный договоры, сотрудничает с ОАО «РЖД»; «нет» - клиент никогда 
не сотрудничал с ОАО «РЖД», не имеет договорных отношений).
